# Spotted Python Week 8 Tutorial
In this script, we aim to practice the following skills:
- Finding photometric redshift
- Integration using multiple methods
- Chemical Abundance
- Initial Mass Function
- Cosmological calculator

In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import astropy.cosmology as cosmo
from astropy.io import fits
import eazy
eazy.fetch_eazy_photoz()
from eazy import photoz

## SED Fitting


### **Example 1:**
AGEL211627-594702 is a strong gravitational lens as part of the AGEL survey sample. Using source detection software, the flux of the sources across multiple bands (g, r, i, z, y) have been extracted and given in the file *AGEL211627-594702_flux.csv*. Below, we will go through an example of how photometric redshifts can be derived using SED fitting.

In [ ]:
# Dictionary of parameters to be used in EAZY
params = {}
params['CATALOG_FILE'] = 'flux.csv'
params['MAIN_OUTPUT_FILE'] = 'output/output.eazypy'

# Galactic extinction
params['MW_EBV'] = 0.0103 #degree of interstellar reddening E(B-V)=(B-V)-(B-V)_0

# Redshift steps
params['Z_STEP'] = 0.01
params['Z_MIN'] = 0.01
params['Z_MAX'] = 2

# Prior
params['PRIOR_ABZP'] = 22.5 # conversion factor from linear flux to magnitudes when flux is in nanomaggies
params['PRIOR_FILTER'] = 28 # K-band
params['PRIOR_FILE'] = 'templates/prior_K_TAO.dat'
params['TEMPLATES_FILE'] = 'templates/fsps_full/tweak_fsps_QSF_12_v3.param'
params['FIX_ZSPEC'] = False
params['IGM_SCALE_TAU'] = 1.0
translate_file = 'translate.csv'

In [ ]:
# Run the photoZ class
self = eazy.photoz.PhotoZ(param_file=None, translate_file=translate_file, zeropoint_file=None,
                          params=params, load_prior=True, load_products=False)

# Fit the full catalog
sample = np.isfinite(self.ZSPEC)
self.fit_catalog(self.idx[sample], n_proc=8)
zout, hdu = self.standard_output(simple=False,
                                 rf_pad_width=0.5, rf_max_err=2,
                                 prior=True, beta_prior=True,
                                 absmag_filters=[],
                                 extra_rf_filters=[])

In [ ]:
# SED Fits
for s in range(0,len(zout['id'])):
    self.OBJID[s]=s
    ix = np.where(self.OBJID == s)[0][0]
    fig, data = self.show_fit(s, xlim=[0.4, 1.2], # wavelength ranges from 0.398 to 1.065 microns
                        show_components=True, add_label=True,
                        template_color='royalblue', logpz=True, zr=[0,params['Z_MAX']])

## Integrating inital mass functions
### **Example 2:**
Given the normalised Miller-Scalo IMF for $0.01\leq M/M_\odot\leq1$ is $$\xi(M)=0.753M^{-1.4}.$$ Calculate the total stellar mass function by integration.

In [ ]:
# Method 1: scipy.integrate
# Define function
def miller_scalo(m):
    return 0.0753*m**-1.4

# define mass range
miller_scalo_lower = 0.01
miller_scalo_upper = 1
miller_scalo_mass = np.linspace(miller_scalo_lower, miller_scalo_upper, 100)

# integrate the function
miller_scalo_scipy = integrate.quad(miller_scalo, miller_scalo_lower, miller_scalo_upper)
miller_scalo_label_scipy = 'Area = ' + str(round(miller_scalo_scipy[0],3))

# plot function and area under
plt.plot(miller_scalo_mass, miller_scalo(miller_scalo_mass), color='darkblue') # plots the function
plt.fill_between(miller_scalo_mass, miller_scalo(miller_scalo_mass), alpha=0.3, color="royalblue", label=miller_scalo_label_scipy) # fills the area
plt.xlabel("$\log_{10} M$")
plt.ylabel("$\log_{10} ξ(M)$")
plt.xscale('log')
plt.yscale('log')
plt.title("Miller-Scalo IMF using scipy.integrate")
plt.legend()
plt.show()

In [ ]:
# Method 2: np.trapz
miller_scalo_trapz = np.trapz(miller_scalo(miller_scalo_mass), miller_scalo_mass)
miller_scalo_label_trapz = 'Area = ' + str(round(miller_scalo_trapz,3))

# plot function and area under
plt.plot(miller_scalo_mass, miller_scalo(miller_scalo_mass), color='red') # plots the function
plt.fill_between(miller_scalo_mass, miller_scalo(miller_scalo_mass), alpha=0.3, color="lightpink", label=miller_scalo_label_trapz) # fills the area
plt.xlabel("$\log_{10} M$")
plt.ylabel("$\log_{10} ξ(M)$")
plt.xscale('log')
plt.yscale('log')
plt.title("Miller-Scalo IMF using np.trapz")
plt.legend()
plt.show()

### **Exercise 1:**
Given the normalised Chabrier IMF for $0.01\leq M/M_\odot\leq1$ is $$\xi(M)=\frac{3.63}{M\ln 10}\exp\left[-\frac{(\log M - \log 0.079)^2}{2(0.69)^2}\right].$$ Select one of the above methods, and calculate the total stellar mass function by integration.

## Integrating Spectral Lines
### **Example 3:**
Equivalent width of an emission lines is given by: $$W_\lambda=\int\left(\frac{F_\lambda}{F_c}-1\right)d\lambda.$$ Using *example_spectra_1.fits* from week 5, calculate the equivalent width of the $H_\alpha$ emission line.

In [ ]:
# open fits file and set variables
hdu_1 = fits.open('example_spectra_1.fits')
header_1 = hdu_1['DATA'].header
flux_1 = hdu_1['DATA'].data
wavelength_1 = header_1['CRVAL1'] + header_1['CDELT1']*np.arange(flux_1.shape[0])

In [ ]:
# focus on the h_alpha line
index_1 = np.where(np.logical_and(wavelength_1<6655, wavelength_1>6640)) # index for wavelength around the h alpha line
wavelength_1_cut = wavelength_1[index_1[0]]
flux_1_cut = flux_1[index_1[0]]
plt.plot(wavelength_1_cut, flux_1_cut, color='royalblue')
plt.xlabel('Wavelength, $\lambda$ $[\mathring{A}]$') # x-axis label
plt.ylabel('Flux, $F$ $[10^{-20} \mathring{A}^{-1} cm^{-2} erg s^{-1}]$') # y-axis label

In [ ]:
# integate spectral line using trapz
continuum_1 = np.median(flux_1) # as an approximate
ew_formula_1 = flux_1_cut/continuum_1-1
ew_1 = np.trapz(ew_formula_1, wavelength_1_cut)
ew_1

### **Exercise 2:**
Using *example_spectra_2.fits* from week 5, calculate the equivalent width of the $H_\alpha$ emission line.

## Cosmology in Python
Astropy.cosmology offers function to calculate distances, ages, lookback times depending on a cosmological model. However, it is limited to the flat $\Lambda CDM$ model.
### **Example 4:**

In [ ]:
cosmo.Planck18.H(0) # hubble constant using parameters from the Planck Collaboration (2020)

In [ ]:
cosmo.WMAP3.H(0.18) # hubble parameter at redshift 0.18 using parameters from pergel et al. (2007) 

In [ ]:
cosmo.WMAP7.comoving_distance(np.array([0.5, 1.0, 1.5])) # comoving distance for objects at redshifts of 0.5, 1.0 and 1.5

In [ ]:
cosmo.Planck13.lookback_time(1.9)